<a href="https://colab.research.google.com/github/eltondeoliveirasilva/tunning/blob/main/tunning_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("/content/drive/MyDrive/churn_data.xlsx")
df.head()

,customerID,tenure,PhoneService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,1,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,34,Yes,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,2,Yes,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,45,No,One year,No,Bank transfer,42.30,1840.75,No
4,9237-HQITU,2,Yes,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7042 entries, 0 to 7041
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7042 non-null   object 
 1   tenure            7042 non-null   int64  
 2   PhoneService      7042 non-null   object 
 3   Contract          7042 non-null   object 
 4   PaperlessBilling  7042 non-null   object 
 5   PaymentMethod     7042 non-null   object 
 6   MonthlyCharges    7042 non-null   float64
 7   TotalCharges      7042 non-null   object 
 8   Churn             7042 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 495.3+ KB


In [4]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7042 entries, 0 to 7041
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7042 non-null   object 
 1   tenure            7042 non-null   int64  
 2   PhoneService      7042 non-null   object 
 3   Contract          7042 non-null   object 
 4   PaperlessBilling  7042 non-null   object 
 5   PaymentMethod     7042 non-null   object 
 6   MonthlyCharges    7042 non-null   float64
 7   TotalCharges      7031 non-null   float64
 8   Churn             7042 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 495.3+ KB


In [6]:
df.isnull().sum()

,0
customerID,0
tenure,0
PhoneService,0
Contract,0
PaperlessBilling,0
PaymentMethod,0
MonthlyCharges,0
TotalCharges,11
Churn,0


In [7]:
df = df.dropna()

In [8]:
df.isnull().sum()

,0
customerID,0
tenure,0
PhoneService,0
Contract,0
PaperlessBilling,0
PaymentMethod,0
MonthlyCharges,0
TotalCharges,0
Churn,0


In [9]:
cat_vars = ['PhoneService','Contract','PaperlessBilling','PaymentMethod']
num_vars = ['tenure','MonthlyChatges','TotalCharges']

modeling_vars = cat_vars + num_vars

In [10]:
x = df.filter(modeling_vars)
y = df['Churn'].copy()
y = pd.DataFrame(y)

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(y.Churn)
y['Churn'] = le.transform(y.Churn)

In [12]:
x = pd.get_dummies(x)

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,stratify= y)

In [27]:
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier()

In [31]:
parameters = {'max_depth':[1,2,3,4,5,6,7,8,9,10],
              'n_estimators':[100,300,500]}

In [32]:
grid_search = GridSearchCV(rf, parameters,scoring= 'roc_auc', cv=5, n_jobs = -1)

In [33]:
grid_search.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': [100, 300, 500]},
             scoring='roc_auc')

In [34]:
results = pd.DataFrame(grid_search.cv_results_)
results.sort_values(by='rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
23,3.467789,0.716974,0.163276,0.013815,8,500,"{'max_depth': 8, 'n_estimators': 500}",0.821362,0.831833,0.807182,0.833769,0.848896,0.828608,0.013858,1
19,2.324250,0.576728,0.125628,0.034453,7,300,"{'max_depth': 7, 'n_estimators': 300}",0.820803,0.829419,0.804323,0.832160,0.853342,0.828009,0.015962,2
21,0.599845,0.014841,0.039784,0.000482,8,100,"{'max_depth': 8, 'n_estimators': 100}",0.820008,0.830210,0.807448,0.832158,0.849415,0.827848,0.013906,3
26,3.495423,0.566729,0.161921,0.008195,9,500,"{'max_depth': 9, 'n_estimators': 500}",0.820813,0.830403,0.809878,0.834183,0.843269,0.827709,0.011461,4
25,2.812814,0.794190,0.159691,0.032808,9,300,"{'max_depth': 9, 'n_estimators': 300}",0.821728,0.830751,0.809016,0.833893,0.843145,0.827707,0.011584,5


In [35]:
grid_search.best_estimator_.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 8,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 500,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [36]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(n_jobs=-1)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [38]:
from sklearn.model_selection import RandomizedSearchCV

parameters = {'leaning_rate':[0.001,0.01],
              'num_leaves':[2,128],
              'min_child_samples':[1,100],
              'subsample':[0.05,1],
              'colsample_bytree':[0.1,1.0]}

In [39]:
random_search = RandomizedSearchCV(lgbm, parameters, scoring='roc_auc', cv=5, n_jobs=-1,n_iter=5)

random_search.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Warning] Unknown parameter: leaning_rate
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: leaning_rate
[LightGBM] [Info] Number of positive: 1486, number of negative: 4138
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006112 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the train set: 5624, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.264225 -> initscore=-1.024125
[LightGBM] [Info] Start training from score -1.024125


RandomizedSearchCV(cv=5, estimator=LGBMClassifier(n_jobs=-1), n_iter=5,
                   n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.1, 1.0],
                                        'leaning_rate': [0.001, 0.01],
                                        'min_child_samples': [1, 100],
                                        'num_leaves': [2, 128],
                                        'subsample': [0.05, 1]},
                   scoring='roc_auc')

In [40]:
results = pd.DataFrame(random_search.cv_results_)
results.sort_values(by='rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_num_leaves,param_min_child_samples,param_leaning_rate,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,5.019370,2.190713,0.045704,0.011816,1,128,1,0.01,1.0,"{'subsample': 1, 'num_leaves': 128, 'min_child...",0.805029,0.814801,0.803427,0.822913,0.816620,0.812558,0.007332,1
0,0.305474,0.240774,0.028632,0.012677,1,2,100,0.001,0.1,"{'subsample': 1, 'num_leaves': 2, 'min_child_s...",0.806826,0.810708,0.785774,0.814813,0.842649,0.812154,0.018239,2
2,1.341851,0.455059,0.035860,0.012627,1,128,100,0.01,0.1,"{'subsample': 1, 'num_leaves': 128, 'min_child...",0.809134,0.807837,0.783507,0.812200,0.837179,0.809971,0.017034,3
4,0.290879,0.064353,0.041639,0.017662,0.05,128,100,0.01,0.1,"{'subsample': 0.05, 'num_leaves': 128, 'min_ch...",0.809134,0.807837,0.783507,0.812200,0.837179,0.809971,0.017034,3
3,0.764796,0.261061,0.041116,0.018873,1,128,1,0.001,0.1,"{'subsample': 1, 'num_leaves': 128, 'min_child...",0.795516,0.797980,0.780338,0.804125,0.821366,0.799865,0.013298,5
